In [32]:
%pip install modelx lifelib -q

Note: you may need to restart the kernel to use updated packages.


Following guide from [lifelib](https://lifelib.io/libraries/notebooks/basiclife/generate_model_points.html)

# [Generating term policy model points with duration](https://lifelib.io/libraries/notebooks/basiclife/generate_model_points_with_duration.html)

In [4]:
### import libraries
# standard libraries
import numpy as np # for math
from numpy.random import default_rng  # Requires NumPy 1.17 or newer
import pandas as pd # for data manipulation
import itertools # for looping

# modelx libraries
import modelx as mx # for model building
import lifelib # for life table

# unique id
import uuid


# Set seed for reproducibility
rng = default_rng(12345)

# Number of Model Points
MPCount = 10000

# Issue Age (Integer): 20 - 59 year old
age_at_entry = rng.integers(low=20, high=60, size=MPCount)

# Sex (Char)
Sex = [
    "M",
    "F"
]

sex = np.fromiter(map(lambda i: Sex[i], rng.integers(low=0, high=len(Sex), size=MPCount)), np.dtype('<U1'))

# Policy Term (Integer): 10, 15, 20
policy_term = rng.integers(low=0, high=3, size=MPCount) * 5 + 10

# Sum Assured (Float): 10000 - 1000000
sum_assured = np.round((1000000 - 10000) * rng.random(size=MPCount) + 10000, -3)

# Duration in month (Int): -36 < Duration(mth) < Policy Term in month
duration_mth = np.rint((policy_term + 3) * 12 * rng.random(size=MPCount) - 36).astype(int)

# Policy Count (Integer): 1
policy_count = np.rint(100 * rng.random(size=MPCount)).astype(int)

In [5]:
# Set attributes
attrs = [
    "age_at_entry",
    "sex",
    "policy_term",
    "policy_count",
    "sum_assured",
    "duration_mth"
]

# Set data
data = [
    age_at_entry,
    sex,
    policy_term,
    policy_count,
    sum_assured,
    duration_mth
]

# Create model point table
model_point_table = pd.DataFrame(dict(zip(attrs, data)), index=range(1, MPCount+1))

# Set index name
model_point_table.index.name = "policy_id"

model_point_table["uuid"] = [str(uuid.uuid1()) for i in range(0, model_point_table.shape[0])]

# shift column 'Name' to first position
first_column = model_point_table.pop('uuid')
  
# insert column using insert(position,column_name,
# first_column) function
model_point_table.insert(0, 'uuid', first_column)

# Display model point table
display(model_point_table)

,uuid,age_at_entry,sex,policy_term,policy_count,sum_assured,duration_mth
policy_id,,,,,,,
1,d3992640-d93b-11ed-a9fe-eedd30f30a66,47,M,10,86,622000.0,1
2,d399279e-d93b-11ed-a9fe-eedd30f30a66,29,M,20,56,752000.0,210
3,d39927bc-d93b-11ed-a9fe-eedd30f30a66,51,F,10,83,799000.0,15
4,d39927d0-d93b-11ed-a9fe-eedd30f30a66,32,F,20,72,422000.0,125
5,d39927e4-d93b-11ed-a9fe-eedd30f30a66,28,M,15,99,605000.0,55
...,...,...,...,...,...,...,...
9996,d39b83a4-d93b-11ed-a9fe-eedd30f30a66,47,M,20,25,827000.0,157
9997,d39b83b8-d93b-11ed-a9fe-eedd30f30a66,30,M,15,81,826000.0,168
9998,d39b83c2-d93b-11ed-a9fe-eedd30f30a66,45,F,20,10,783000.0,146


# Additional Component for IAA preso

In [23]:
uuid_extension = pd.Series([str(uuid.uuid1()) for i in range(0, model_point_table.shape[0])])

underwriter_comments = pd.DataFrame()
underwriter_comments["uuid"] = model_point_table.uuid.append(uuid_extension)
underwriter_comments.index.name = "policy_id"


# create new pd column and randomly assign values
underwriter_comments["discount"] = np.random.choice([True, False], underwriter_comments.shape[0])

underwriter_comments.reindex()
display(underwriter_comments)

ValueError: cannot reindex on an axis with duplicate labels

# [Create Premium Table](https://lifelib.io/libraries/notebooks/basiclife/create_premium_table.html)

In [50]:
# set to True if you want to remove existing cluster folder
REMOVE_CLUSTER = True

#check if there is a folder named "cluster" in the current directory
if os.path.exists("cluster") & REMOVE_CLUSTER:
    shutil.rmtree("cluster")

lifelib.create("cluster", r"./cluster")

In [45]:
space.model_point_table = model_point_table

In [46]:
@mx.defcells
def premium_rate():
    return (1 + loading_prem()) * net_premium_pp()

In [47]:
model_point_table['premium'] = space.premium_rate()

# Distorting data for preso

In [ ]:
#Add noise
for col in model_point_table.columns:
    model_point_table.loc[model_point_table.sample(frac=0.1).index, col] = pd.np.nan

In [ ]:
distorted_names = ['UUid ',
 'Age At Entry',
 'Sex ',
 'Policy Term  ',
 'Policy Count',
 'Sum Assured ',
 'Issue date',
 'Payment freq',
 'Payment_term',
 'Premium']

rename_dict = dict()
for i in range(0,len(distorted_names)):
  rename_dict.update({list(model_point_table.columns)[i]:distorted_names[i]})
  
model_point_table = model_point_table.rename(columns=rename_dict)
model_point_table.columns

In [ ]:
model_point_table.to_excel("model_point.xlsx")

In [ ]:
space.premium_pp.formula

In [ ]:
df = pd.DataFrame(itertools.product(range(20, 60), [10, 15, 20]), columns=['age_at_entry', 'policy_term'])
df.index += 1
df.index.name = 'point_id'
df["sum_assured"] = 1
df

In [ ]:
len(space.premium_rate())

In [ ]:
premium_rate_table = df.copy()
space.model_point_table = df
premium_rate_table["premium_rate"] = space.premium_rate()
del premium_rate_table["sum_assured"]
premium_rate_table.set_index(["age_at_entry", "policy_term"], inplace=True)
premium_rate_table["premium_rate"]   # As Series

In [ ]:
model_point_table